In [58]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.client import device_lib
import os, re, json
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
import logging

from keras.layers import Input, Dense, GlobalMaxPooling2D, Flatten, Concatenate
from keras.models import Model, Sequential

In [3]:
tf.config.list_physical_devices()
tf.debugging.set_log_device_placement(True)

In [62]:
# Set the TensorFlow logger to ERROR level
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [4]:
DATA_PATH = os.path.abspath(os.path.join("../../..", "Seafile", "Detecting-Objects", "DataCollection"))
IMAGE_PATH = os.path.join(DATA_PATH, "images")
LABEL_PATH = os.path.join(DATA_PATH, "labels")

TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
VAL_PATH = os.path.join(DATA_PATH, 'val')

In [7]:
def load_image(path):
    img = cv2.imread(path)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)
    rgb_tensor = tf.expand_dims(rgb_tensor, 0)
    
    return rgb_tensor

In [36]:
# Load the pre-trained model from a TensorFlow Hub module
MODEL_URL = "https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/1"
INPUT_SHAPE = (512, 512, 3)

eff_det = hub.KerasLayer(
    handle=MODEL_URL,
    input_shape=INPUT_SHAPE,
    trainable=False
)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

...

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

In [41]:
eff_det.trainable = False

In [53]:
input_layer = Input(shape=INPUT_SHAPE)

#
class_output = eff_det(input_layer, training=False)
class_output = GlobalMaxPooling2D()(class_output)
class_output = Dense(2048, activation='relu')(class_output)
class_output = Dense(3, activation='softmax')(class_output)

# Apply the pre-trained model to the input tensor again
regress_output = eff_det(input_layer, training=False)
regress_output = GlobalMaxPooling2D()(regress_output)
regress_output = Dense(2048, activation='relu')(regress_output)
regress_output = Dense(4, activation='sigmoid')(regress_output)

# Create a functional model using the input and output tensors
model = Model(inputs = input_layer, outputs=[class_output, regress_output])

ValueError: Layer "global_max_pooling2d" expects 1 input(s), but it received 10 input tensors. Inputs received: [<tf.Tensor 'Placeholder:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(None, 4, 4, 224) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(None, 4, 4, 224) dtype=float32>]

In [74]:
# Define the input layer
inputs = Input(shape=(512, 512, 3))

# Pass the input through the eff_det layer to get the feature vector
features = eff_det(inputs)

# Use a dense layer to perform classification on the feature vector
classification = Dense(units=10, activation="softmax")(features)

# Use another dense layer to predict bounding boxes on the feature vector
bounding_boxes = Dense(units=4)(features)

# Create a model using the functional API and compile it
model = keras.Model(inputs=inputs, outputs=[classification, bounding_boxes])

'''
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=[keras.losses.SparseCategoricalCrossentropy(), keras.losses.MeanSquaredError()],
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model on some data
model.fit(X_train, [y_train_classification, y_train_bounding_boxes], epochs=10)
'''

ValueError: Layer "dense_15" expects 1 input(s), but it received 10 input tensors. Inputs received: [<tf.Tensor 'Placeholder:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(None, 4, 4, 224) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(None, 4, 4, 224) dtype=float32>]

In [75]:
#x = eff_det(input_layer)
#x = eff_det(include_top = False)(input_layer)
input_layer = Input(shape=(512, 512, 3))

# Apply the pre-trained model to the input tensor
class_output = eff_det(input_layer)
class_output = Dense(2048, activation='relu')(class_output)
class_output = Dense(3, activation='softmax')(class_output)

# Apply the pre-trained model to the input tensor again
regress_output = eff_det(input_layer)
regress_output = Dense(2048, activation='relu')(regress_output)
regress_output = Dense(4, activation='sigmoid')(regress_output)

# Create a functional model using the input and output tensors
model = Model(inputs = input_layer, outputs=[class_output, regress_output])

ValueError: Layer "dense_16" expects 1 input(s), but it received 10 input tensors. Inputs received: [<tf.Tensor 'Placeholder:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(None, 4, 4, 224) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(None, 4, 4, 224) dtype=float32>]

In [57]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
efficient_det = "https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/1"

feature_extractor_model = efficient_det

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(512, 512, 3),
    trainable=False
)

#feature_batch = feature_extractor_layer(image_batch)
#print(feature_batch.shape)

num_classes = 3

# Define the input layer
inputs = Input(shape=(512, 512, 3))

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

...

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in dev

ValueError: Layer "dense_10" expects 1 input(s), but it received 10 input tensors. Inputs received: [<tf.Tensor 'Placeholder:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(None, 4, 4, 224) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(None, 4, 4, 224) dtype=float32>]

In [69]:
# Apply the pre-trained feature extractor to the input tensor
x = feature_extractor_layer(inputs)

# Add a Dense layer with `num_classes` output units
outputs = Dense(num_classes)(x)

# Create a functional model using the input and output tensors
model = Model(inputs=inputs, outputs=outputs)

model.summary()

ValueError: Layer "dense_14" expects 1 input(s), but it received 10 input tensors. Inputs received: [<tf.Tensor 'Placeholder:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(None, 4, 4, 224) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(None, 64, 64, 224) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(None, 32, 32, 224) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(None, 16, 16, 224) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(None, 8, 8, 224) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(None, 4, 4, 224) dtype=float32>]